#**Aula 1: Realizando as primeiras consultas**

##**Conhecendo os dados**

In [9]:
url = 'https://raw.githubusercontent.com/alura-cursos/llamaIndex_pandas_query/refs/heads/main/Dados/vendas.csv'

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv(url)

In [12]:
df.head()

,ID_compra,filial,cidade,tipo_cliente,genero,tipo_produto,preco_unitario,quantidade,imposto_5%,total,data,hora,forma_pagamento,avaliacao
0,750-67-8428,A,Santo André,Membro,Feminino,Saúde e Beleza,74.69,7,26.1415,548.9715,2024-01-05,13:08:00,Carteira Digital,9.1
1,226-31-3081,C,São Caetano,Normal,Feminino,Eletrônicos,15.28,5,3.8200,80.2200,2024-03-08,10:29:00,Dinheiro,9.6
2,631-41-3108,A,Santo André,Normal,Masculino,Casa,46.33,7,16.2155,340.5255,2024-03-03,13:23:00,Cartão de Crédito,7.4
3,123-19-1176,A,Santo André,Membro,Masculino,Saúde e Beleza,58.22,8,23.2880,489.0480,2024-01-27,20:33:00,Carteira Digital,8.4
4,373-73-7910,A,Santo André,Normal,Masculino,Esportes e Viagem,86.31,7,30.2085,634.3785,2024-02-08,10:37:00,Carteira Digital,5.3


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID_compra        1000 non-null   object 
 1   filial           1000 non-null   object 
 2   cidade           1000 non-null   object 
 3   tipo_cliente     1000 non-null   object 
 4   genero           1000 non-null   object 
 5   tipo_produto     1000 non-null   object 
 6   preco_unitario   1000 non-null   float64
 7   quantidade       1000 non-null   int64  
 8   imposto_5%       1000 non-null   float64
 9   total            1000 non-null   float64
 10  data             1000 non-null   object 
 11  hora             1000 non-null   object 
 12  forma_pagamento  1000 non-null   object 
 13  avaliacao        1000 non-null   float64
dtypes: float64(4), int64(1), object(9)
memory usage: 109.5+ KB


In [14]:
df.duplicated().sum()

np.int64(0)

##**Definindo a LLM e configurando a Chave de API**

In [15]:
!pip install llama-index llama-index-experimental -q

In [16]:
from google.colab import userdata
key = userdata.get('GROQ_API')

In [18]:
!pip install llama-index-llms-groq -q

In [23]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

Settings.llm = Groq(model='llama-3.3-70b-versatile', api_key=key)

##**Gerando consultas com PandasQueryEngine**

In [24]:
from llama_index.experimental.query_engine import PandasQueryEngine

In [25]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [26]:
response = query_engine.query('Qual é a forma de pagamento mais utilizada pelos clientes?')

> Pandas Instructions:
```
df['forma_pagamento'].mode()[0]
```
> Pandas Output: Carteira Digital


In [27]:
df.forma_pagamento.value_counts()

,count
forma_pagamento,
Carteira Digital,345
Dinheiro,344
Cartão de Crédito,311


In [28]:
response = query_engine.query('Qual é o tipo de produto com maior quantidade por filial?')

> Pandas Instructions:
```
df.groupby(['filial', 'tipo_produto'])['quantidade'].sum().groupby('filial').idxmax()
```
> Pandas Output: filial
A                   (A, Casa)
B      (B, Esportes e Viagem)
C    (C, Alimentos e Bebidas)
Name: quantidade, dtype: object


In [29]:
df.groupby(['filial', 'tipo_produto'])['quantidade'].sum()

filial  tipo_produto       
A       Alimentos e Bebidas    313
        Casa                   371
                              ... 
C       Moda                   342
        Saúde e Beleza         277
Name: quantidade, Length: 18, dtype: int64

In [30]:
response

Response(response='filial\nA                   (A, Casa)\nB      (B, Esportes e Viagem)\nC    (C, Alimentos e Bebidas)\nName: quantidade, dtype: object', source_nodes=[], metadata={'pandas_instruction_str': "df.groupby(['filial', 'tipo_produto'])['quantidade'].sum().groupby('filial').idxmax()", 'raw_pandas_output': 'filial\nA                   (A, Casa)\nB      (B, Esportes e Viagem)\nC    (C, Alimentos e Bebidas)\nName: quantidade, dtype: object'})

In [31]:
print(response.response)

filial
A                   (A, Casa)
B      (B, Esportes e Viagem)
C    (C, Alimentos e Bebidas)
Name: quantidade, dtype: object


In [32]:
response = query_engine.query('Qual é a forma de pagamento mais utilizada pelos clientes em cada filial?')
response

> Pandas Instructions:
```
df.groupby('filial')['forma_pagamento'].apply(lambda x: x.mode().iloc[0])
```
> Pandas Output: filial
A    Carteira Digital
B    Carteira Digital
C            Dinheiro
Name: forma_pagamento, dtype: object


Response(response='filial\nA    Carteira Digital\nB    Carteira Digital\nC            Dinheiro\nName: forma_pagamento, dtype: object', source_nodes=[], metadata={'pandas_instruction_str': "df.groupby('filial')['forma_pagamento'].apply(lambda x: x.mode().iloc[0])", 'raw_pandas_output': 'filial\nA    Carteira Digital\nB    Carteira Digital\nC            Dinheiro\nName: forma_pagamento, dtype: object'})

In [33]:
response = query_engine.query('Em quais cidades temos filiais?')
response

> Pandas Instructions:
```
df['cidade'].unique()
```
> Pandas Output: ['Santo André' 'São Caetano' 'São Bernardo do Campo']


Response(response="['Santo André' 'São Caetano' 'São Bernardo do Campo']", source_nodes=[], metadata={'pandas_instruction_str': "df['cidade'].unique()", 'raw_pandas_output': "['Santo André' 'São Caetano' 'São Bernardo do Campo']"})

In [34]:
response = query_engine.query('Qual é a filial com maior faturamento?')
response

> Pandas Instructions:
```
df.groupby('filial')['total'].sum().idxmax()
```
> Pandas Output: C


Response(response='C', source_nodes=[], metadata={'pandas_instruction_str': "df.groupby('filial')['total'].sum().idxmax()", 'raw_pandas_output': 'C'})